# Data Loading, Preparation, and Exploration

References:

[1] VanderPlas, Jake. *Python data science handbook: Essential tools for working with data. Second Edition* " O'Reilly Media, Inc.", 2023.

[2] McKinney, Wes. *Python for data analysis.* " O'Reilly Media, Inc.", 2022.

[3] Johansson, Robert, Robert Johansson, and Suresh John. *Numerical python.* Vol. 1. New York: Apress, 2019.

In [5]:
import numpy as np
import pandas as pd

## 1 Data Loading

Reading the data is most of the time, the first step in any data analysis. We also use a similar term *parsing* when we describe loading text data and interpreting it as tables and into different data types. In this notebook, we'll cover the basics of data loading using pandas.

### Reading `csv` files

Let's start with reading a small comma-separated values (CSV):

In [69]:
!cat data/ex1.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


Since this data is comma delimited, we can readily use `pandas.read_csv` to read and turn it into a `DataFrame`.

In [81]:
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Some files don't have a header (column label) row.

In [83]:
!cat data/ex2.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In such cases, we can let pandas define a default integer columns,

In [85]:
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Or we can specify the names ourselves:

In [89]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In the above example, pandas creates a default integer index for each row of the data. If we want to let one of the columns in the csv become our index, we can specifically set it using the `index_col` parameter.

In [93]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Other files can also have other delimeters, using whitespace or other patterns to separate the fields.

In [95]:
!cat ex3.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In this case the file is separated by tabs. We can specify the delimeter used in the file using the `sep` or `delimiter` parameter.

In [97]:
pd.read_csv('ex3.txt', sep='\t')

,A,B,C
aaa,0.262025,0.158684,0.278127
bbb,0.459317,0.321001,0.518393
ccc,0.261943,0.976085,0.732815
ddd,0.115274,0.386275,0.628501


The `sep` parameter can also be specified using a regular expression syntax.

The file parsing function also has additional parameters which lets you handle a wide variety of exception file formats. For example, we can skip specific rows in the text file using the `skiprows` parameter.

In [ ]:
!cat data/ex4.csv

In [99]:
pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Parsing null values

pandas also handles missing values automatically if it recognizes an empty string or a denoted *sentinel* (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:

In [ ]:
!cat data/ex5.csv

Here are the list of the default null sentinels recognized by pandas:

In [101]:
pd._libs.parsers.STR_NA_VALUES

{'',
 '#N/A',
 '#N/A N/A',
 '#NA',
 '-1.#IND',
 '-1.#QNAN',
 '-NaN',
 '-nan',
 '1.#IND',
 '1.#QNAN',
 '<NA>',
 'N/A',
 'NA',
 'NULL',
 'NaN',
 'None',
 'n/a',
 'nan',
 'null'}

In [103]:
pd.read_csv('ex5.csv')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


We can use `pd.isna` to check which values in the dataframe are null (or `NaN`).

In [105]:
pd.isna(pd.read_csv('ex5.csv'))

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


If you want to disable the default sentinel values being used by pandas, you can specify the `keep_default_na` to be `False`.

In [107]:
pd.read_csv('ex5.csv', keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


Then add using the `na_values` parameter the sentinel null values that you want to set as `NaN`.

In [109]:
pd.read_csv('ex5.csv', keep_default_na=False, na_values=['NA'])

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


You can also set different null sentinels for each column using a dictionary

In [ ]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data/ex5.csv', keep_default_na=False, na_values=sentinels)

### Reading text files in pieces

If we are given a large text file, pandas has ooptions avaiable to read it in smaller pieces.

In [123]:
df = pd.read_csv('ex6.csv')
df

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


We can read a small number of rows by specifying `nrows`

In [111]:
pd.read_csv('ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


We can also read the file by chunk through specification of the `chunksize` parameter:

In [113]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

In [115]:
type(chunker)

pandas.io.parsers.readers.TextFileReader

This text file reader allows us to iterate the parts of the file according to the `chunksize`. For example, we can iterate to our data while aggregating the value counts in the `key` column.

In [117]:
# Create text file reader with specified chunksize
chunker = pd.read_csv('ex6.csv', chunksize=1000)

# Iterate on the chunker while aggregating the result container
total = pd.Series([], dtype='int64')
for piece in chunker:
    total = total.add(piece['key'].value_counts(), fill_value=0)

In [119]:
total

key
0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
A    320.0
B    302.0
C    286.0
D    320.0
E    368.0
F    335.0
G    308.0
H    330.0
I    327.0
J    337.0
K    334.0
L    346.0
M    338.0
N    306.0
O    343.0
P    324.0
Q    340.0
R    318.0
S    308.0
T    304.0
U    326.0
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
dtype: float64

### Writing Data to a Text File

We can also export data to a delimited format.

In [125]:
data = pd.read_csv('ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using the `DataFrame`'s `to_csv` method, we can write the data to an output comma-separated file.

In [127]:
data.to_csv('out.csv')

In [131]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


We can also use other delimiters by specifying the `sep` parameter

In [133]:
data.to_csv('out.csv', sep='|')

In [135]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


Missing values appear as strings in the output, if we want to denote null values using other sentinel value, we can set the `na_rep` parameter.

In [137]:
data.to_csv('out.csv', na_rep='NULL')

In [139]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


We can also opt to not include the row and column index by specifying the `index` and `header` parameter respectively.

In [141]:
data.to_csv('out.csv', index=False, header=False)

In [143]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


We can also write a subset of columns in the order of your choosing

In [145]:
data.to_csv('out.csv', index=False, columns=['a', 'b', 'c'])

In [147]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


We can set the write `mode` parameter if we want to instead append values to the text file.

In [149]:
data.to_csv('out.csv', index=False, columns=['a', 'b', 'c'], mode='a',
            header=False)

In [151]:
!cat out.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


### Binary Data Formats

We can also serialize a data frame in `pickle` format using pandas built in `to_pickle` method.

In [153]:
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [155]:
df.to_pickle('df.pickle')

Pickle files are generally readable only in Python. You can read any "pickled" object stored in a file using the `pickle` module.

In [158]:
import pickle

In [160]:
with open('df.pickle', 'rb') as f:
    pickled_df = pickle.load(f)
pickled_df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Or conveniently using the `pd.read_pickle` function:

In [162]:
pd.read_pickle('df.pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 2 Data Preparation

In any data science project, the data preparation: loading, cleaning, transforming, and rearranging takes up 80% of time. Here we explore data preparation methods that is readily available to us in pandas.

### Handling Missing Data

For data with `float64` dtype, pandas using a floating-point value `NaN` to represent missing data.

In [164]:
float_data = pd.Series([1.2, -3.5, np.nan, None, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    NaN
4    0.0
dtype: float64

Notice that both the numpy `nan` value and `None` is recognized by pandas as null values.

In [166]:
float_data.isna()

0    False
1    False
2     True
3     True
4    False
dtype: bool

#### Filtering Out Missing Data

There are few ways in which we can filter missing data.

In [168]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

We can use the `dropna` method.

In [170]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Or index not null values.

In [172]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With `DataFrame`s, you may want to drop rows or columns that are all null, or only those rows containing any null values. `dropna` by default drops any row containing a missing value.

In [174]:
data = pd.DataFrame([[1., 6.5, 6.],
                     [2., np.nan, np.nan],
                     [np.nan, np.nan, np.nan],
                     [np.nan, 7.5, 3.]])
data

,0,1,2
0,1.0,6.5,6.0
1,2.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,7.5,3.0


In [176]:
data.dropna()

,0,1,2
0,1.0,6.5,6.0


To drop only rows that are all nulls, we specify the `all` to the `how` parameter.

In [179]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,6.0
1,2.0,NaN,NaN
3,NaN,7.5,3.0


To drop with respect to the columns instead, we specify the `axis` parameter.

In [182]:
data.dropna(axis='columns')

""
0
1
2
3


In [184]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,6.0,NaN
1,2.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,7.5,3.0,NaN


In [186]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,6.0
1,2.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,7.5,3.0


If we want to keep only rows containing at most a certain number of missing observations. We can indicate this using the `thresh` argument.

In [189]:
data

,0,1,2,4
0,1.0,6.5,6.0,NaN
1,2.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,7.5,3.0,NaN


In [191]:
data.dropna(thresh=2)

,0,1,2,4
0,1.0,6.5,6.0,NaN
3,NaN,7.5,3.0,NaN


#### Filling In Missing Data

Rather than filtering out missing data, we may want to fill the null values in a number of ways. For most cases, the `fillna` method allows us to use a constant to replace missing values.

In [195]:
data.fillna(-10)

,0,1,2,4
0,1.0,6.5,6.0,-10.0
1,2.0,-10.0,-10.0,-10.0
2,-10.0,-10.0,-10.0,-10.0
3,-10.0,7.5,3.0,-10.0


We can also use a dictionary to specify null values for each column.

In [198]:
data.fillna({0: -10, 1: -100, 2: -1_000, 4: -10_000})

,0,1,2,4
0,1.0,6.5,6.0,-10000.0
1,2.0,-100.0,-1000.0,-10000.0
2,-10.0,-100.0,-1000.0,-10000.0
3,-10.0,7.5,3.0,-10000.0


We can also do the forward filling and backfilling method which we encountered in the previous notebook.

In [201]:
data.fillna(method='ffill')

C:\Users\kate\AppData\Local\Temp\ipykernel_13660\1844443866.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill')


,0,1,2,4
0,1.0,6.5,6.0,NaN
1,2.0,6.5,6.0,NaN
2,2.0,6.5,6.0,NaN
3,2.0,7.5,3.0,NaN


In [203]:
data.fillna(method='bfill')

C:\Users\kate\AppData\Local\Temp\ipykernel_13660\1397359497.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill')


,0,1,2,4
0,1.0,6.5,6.0,NaN
1,2.0,7.5,3.0,NaN
2,NaN,7.5,3.0,NaN
3,NaN,7.5,3.0,NaN


One of the common ways of imputing a value in the data is to use the mean value of that column.

In [206]:
data.fillna(data.mean())

,0,1,2,4
0,1.0,6.5,6.0,NaN
1,2.0,7.0,4.5,NaN
2,1.5,7.0,4.5,NaN
3,1.5,7.5,3.0,NaN


### Data Transformation

#### Removing Duplicates

Given a data with duplicates,

In [208]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


We can first determine which rows are duplicates of one another using the `duplicated` method.

In [211]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Or drop them directly using the `drop_duplicates` method.

In [214]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both by default consider both columns. Alternatively, we can specify any subset to drop duplicated rows across that subset.

In [217]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [219]:
data.drop_duplicates(subset=['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


By default, the first occurrence is being retained. Passing `last` to the `keep` parameter will return the last one.

In [222]:
data.drop_duplicates(subset=['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


#### Transforming Data Using a Function or Mapping

We can also perform some transformation based on values in an array, Series, or a column in the dataframe. Consider the following hypothetical data collected about various kind of meat:

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Suppose we want to add a column indicating the type of animal that each food came from. We can do this by defining a mapping between each distinct meat type to the kind of animal.

In [ ]:
meat_to_animal = {
"bacon": "pig",
"pulled pork": "pig",
"pastrami": "cow",
"corned beef": "cow",
"honey ham": "pig",
"nova lox": "salmon"
}

In [ ]:
data['animal'] = data['food'].map(meat_to_animal)
data

Alternatively, we can use a function to perform and transform the values of the data frame. For example, we want to convert the ounce values to kilograms, we can define the conversion:

In [ ]:
ounces_to_kg = lambda x: 0.0283495*x

In [ ]:
data['kg'] = data.ounces.apply(ounces_to_kg)
data

#### Replacing Values

We can also replace specific values using the `replace` method.

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

We can specify this in various ways

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

#### Renaming Axis Indexes

Axis labels can also be trasnforemed by a funciton or mapping to produce new, differently labeled objects.

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

We can assign the `index` of a data frame to modify it in place.

In [ ]:
data.index = data.index.map(lambda x: x[:4].upper())
data

If we want to create a transformed version without modifying the original, we can use the `rename` method.

In [ ]:
data.rename(index=str.title, columns=str.upper)

We can also specify a dictionary instead of a function

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

#### Discretization and Binning

Continuous data is often discretized or separated into "bins" for analysis. For example, we have data about age of a population and you want to group them into a discrete age buckets:

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

In [ ]:
age_categories = pd.cut(ages, bins)
age_categories

The result is a special `Categorical` object. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin.

In [ ]:
age_categories.codes

In [ ]:
age_categories.categories

In [ ]:
age_categories.categories[0]

We can specify which side is closed by passing the `right` argument.

In [ ]:
pd.cut(ages, bins, right=False)

You can override the default interval-based labeling by passing a list or array to the `labels` parameter.

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to `pandas.cut` it will compute an equal-length bin based on the minimum and maximum values in the data.

In [ ]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

`precision` specifies the decimal precision of the output.

A closely related function is `pd.qcut` which bins the data based on sample quantiles.

In [ ]:
data = np.random.standard_normal(1_000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

In [ ]:
pd.value_counts(quartiles)

#### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Suppose give a data,

In [ ]:
data = pd.DataFrame(np.random.standard_normal((1_000, 4)))
data

We want to retain only those within 3 sigma of the mean

$$
x = \mu \pm 3 \sigma
$$

We can use a combination of boolean array operations to determine which values are outliers.

In [ ]:
outliers_mask = data > data.mean() + 3*data.std()
outliers_mask |= data < data.mean() - 3*data.std()

In [ ]:
outliers_mask.sum()

To select the rows having values that exceeds our outlier condition, we can use  the `any` method on a Boolean DataFrame.

In [ ]:
data[outliers_mask.any(axis='columns')]

#### Permutation and Random Sampling

Permuting (randomly reordering) a Series or DataFrame is possible using the `np.random.permutation` function. Afterwards, we can use fancy indexing or the `take` method to get the permuted data.

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

In [ ]:
np.random.seed(1337)
sampler = np.random.permutation(df.shape[0])
sampler

In [ ]:
df.iloc[sampler]

In [ ]:
df.take(sampler)

We can also permute the columns by specifying the `axis` parameter.

In [ ]:
np.random.seed(1337)
column_sampler = np.random.permutation(df.shape[1])
df.take(column_sampler, axis='columns')

### String Manipulation

pandas incorporates the usual built-in python string operations as a methods operating on a Series. Here is a partial list of those methods.

<img src="images/string.png" style="width: 65%;">

We will demonstrate a few of those using this sample data.

In [ ]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data

In [ ]:
data.str.contains('gmail')

In [ ]:
data.str.split('@')

In [ ]:
data.str.split('@').str.join('#')

In [ ]:
data.str.split('@').str[0]

## Data Exploration

pandas object are also equipped with a set of common mathematical and statistical methods that allow for reduction or summarization of a dataset.

Say we are given with the following dataset:

In [ ]:
data = pd.read_csv(
    'data/2022-2023_nba_regular.csv', sep=';', encoding = 'ISO-8859-1',
).drop_duplicates(subset='Rk').set_index('Rk')
data

### Summary Statistics

We can use the `describe` method to give a summary statistics for chosen columns:

In [ ]:
data.columns

In [ ]:
columns = ['AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']
data.loc[:, columns].describe()

We can reduce the dataset using common summary statistics and related methods.

In [ ]:
data.loc[:, columns].median()

In [ ]:
data.loc[:, columns].max()

In [ ]:
data.loc[:, columns].min()

### Correlation and Covariance

You can also compute the correlation per series or per data frame

In [ ]:
data['3P'].corr(data.TOV)

In [ ]:
data.loc[:, columns].corr()

In [ ]:
data.loc[:, columns].cov()

### Unique Values, Value Counts, and Membership

Another class of method extracts information about the unique values contained in a Series or DataFrame. For example, we want to get the unique positions in NBA, we can use the `unique` method.

In [ ]:
data.Pos.unique()

A very common operation is to compute the frequency counts of a specific variable. For example, if we want to compute the frequency counts of the age in our dataset, we can:

In [ ]:
age_counts = data.Age.value_counts().sort_index()
age_counts

We can also generate a frequency plot count using this data.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

ax.bar(age_counts.index, age_counts)
ax.spines[['top', 'right']].set_visible(False)

More visualization options will be discussed in the 

## 4 Supplemental Exercises

For additional exercises, check the following resources:

1. https://codesolid.com/pandas-practice-examples/#part-two-pandas-review-questions
2. https://www.practiceprobs.com/problemsets/python-pandas/
3. https://www.w3resource.com/python-exercises/pandas/index.php